# Check for cic's that sent stats to S3 but are not linked to an ordernumber.
These cic's are potentially accidentally removed from our cloud during refurbishing of a batch of cics.

In [1]:
import mysql.connector
from mysql.connector import Error
from urllib.parse import urlparse
from dotenv import load_dotenv
import os
from pathlib import Path
import json
import redis
import pandas as pd # only if needed for filtering of data

In [2]:
# load .env file
env_path = Path('../') / '.env'
load_dotenv(env_path)

REDIS_URL = os.getenv("REDISPROD")
MYSQL_URL = os.getenv('MYSQLPROD')

In [3]:
def get_cic_stats_from_redis(redis_url):
    # Connect to Redis database
    parsed_url = urlparse(redis_url)
    r = redis.Redis(host=parsed_url.hostname,
                    port=parsed_url.port,
                    db=0,
                    password=parsed_url.password,
                    username=parsed_url.username)
    
    # get objects from redis
    redis_objects = r.mget(r.keys(pattern="cic:*CIC*lastStat*"))
    results = []
    for obj in redis_objects:
        try:
            results.append(json.loads(obj.decode()))
        except:
            pass
    return results

In [4]:
# get data from mysql
parsed_mysql_url = urlparse(MYSQL_URL)

try:
    connection = mysql.connector.connect(host=parsed_mysql_url.hostname,
                                         user=parsed_mysql_url.username,
                                         password=parsed_mysql_url.password,
                                         database=parsed_mysql_url.path[1:],
                                         port=parsed_mysql_url.port)
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM cic WHERE orderNumber is NULL;")
        fields = [field_md[0] for field_md in cursor.description]
        result = [dict(zip(fields,row)) for row in cursor.fetchall()]
        df_mysql = pd.DataFrame(result)
except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.28
MySQL connection is closed


In [5]:
# get cic data from redis
redis_data = get_cic_stats_from_redis(REDIS_URL)

df_redis = pd.json_normalize(redis_data)

In [6]:
df_mysql

,id,createdAt,updatedAt,addressNumber,addressStreet,availableWifiNetworks,isScanningForWifi,lastScannedForWifi,updateStatus,updateUntil,...,maximumHeatingOutdoorTemperature,ratedMaximumHousePower,orderNumber,status,lteInfo,hubspotDealId,name,usePricingToLimitHeatPump,installationId,numberOfHeatPumps
0,CIC-002fd9e7-4ec3-5ba0-9887-8200ac406ae9,2023-07-27 04:44:46.780,2023-07-27 04:44:47.562,None,None,None,0,NaT,None,None,...,NaN,NaN,None,active,{},None,None,1,1811.0,NaN
1,CIC-00397c56-d471-5a15-a964-56191a52c44f,2023-03-20 08:46:33.360,2023-04-01 11:08:14.036,None,None,None,0,NaT,None,None,...,NaN,NaN,None,active,{},None,None,1,NaN,NaN
2,CIC-0056683f-8a2f-5fe0-bb22-d8441ffa81f5,2023-07-27 17:17:31.450,2023-07-27 17:17:32.252,None,None,None,0,NaT,None,None,...,NaN,NaN,None,active,{},None,None,1,1872.0,NaN
3,CIC-005e12d2-099f-5168-a84a-6ddb6bb369c3,2023-01-11 13:21:27.074,2023-01-26 12:14:20.457,None,None,None,0,NaT,None,None,...,NaN,NaN,None,active,{},None,None,1,NaN,NaN
4,CIC-006b95f2-4bd3-5f5d-acab-8bedc3bdfbff,2023-07-25 01:40:13.754,2023-07-25 01:40:14.601,None,None,None,0,NaT,None,None,...,NaN,NaN,None,active,{},None,None,1,1549.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1528,CIC-ff9be4e4-38c8-59bd-a8eb-2eeee2922631,2023-07-21 05:40:19.162,2023-07-21 05:40:20.056,None,None,None,0,NaT,None,None,...,NaN,NaN,None,active,{},None,None,1,1149.0,NaN
1529,CIC-ffb65bfc-aa61-5b67-aee9-e31e71b63daa,2023-07-25 06:22:27.855,2023-07-25 06:22:28.495,None,None,None,0,NaT,None,None,...,NaN,NaN,None,active,{},None,None,1,1576.0,NaN
1530,CIC-ffda3016-6776-50e0-9e34-da189ee90622,2023-07-20 12:28:03.665,2023-07-31 07:42:27.601,None,None,None,0,NaT,None,None,...,NaN,NaN,None,active,{},None,None,1,1066.0,NaN
1531,localtest,2022-11-04 10:52:08.856,2022-11-04 15:46:18.580,None,None,None,0,NaT,None,None,...,NaN,NaN,None,registering,{},None,None,1,NaN,NaN


In [15]:
# find ids that are in redis but not in mysql
mysql_ids = df_mysql['id'].tolist()
df_redis[(df_redis['system.quattId'].isin(mysql_ids))][['system.quattId', 'time.tsHuman', 'qc.supervisoryControlMode']].sort_values(by='system.quattId').dropna(subset=['qc.supervisoryControlMode'])


,system.quattId,time.tsHuman,qc.supervisoryControlMode
610,CIC-d2e43588-fe92-5f82-a91c-98af4b9eea2b,2023-08-01T06:55:44.616Z,0.0
1104,CIC-d7e946f0-eedc-5972-895b-7460a06ece1e,2023-08-01T06:55:48.897Z,0.0


In [17]:
df_mysql[df_mysql['id'].str.contains('d2e43')]
df_redis[df_redis['system.quattId'] == "CIC-d2e43588-fe92-5f82-a91c-98af4b9eea2b"]

,hp2,time.ts,time.tsHuman,hp1.modbusSlaveId,hp1.setMainWorkingMode,hp1.setCompressorFrequencyLevel,hp1.setCirculatingPumpRelay,hp1.setCirculatingPumpDutyCycle,hp1.waterPumpLevel,hp1.getMainWorkingMode,...,system.externalEnergyMeter1Flow,system.externalEnergyMeter1Supply,system.externalEnergyMeter1Return,system.externalEnergyMeter1EnergyCooling,system.externalEnergyMeter2Energy,system.externalEnergyMeter2Power,system.externalEnergyMeter2Flow,system.externalEnergyMeter2Supply,system.externalEnergyMeter2Return,system.externalEnergyMeter2EnergyCooling


In [18]:
len(df_redis)

2243